In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [3]:
train_X=pd.read_csv("/kaggle/input/2021-ai-w6-p1/pca_train.csv")
train_Y=pd.read_csv("/kaggle/input/2021-ai-w6-p1/train_label.csv").label
test_X=pd.read_csv("/kaggle/input/2021-ai-w6-p1/pca_test.csv")

In [21]:
train_X.head(3)

In [22]:
train_Y.head(3)

In [23]:
x_train, x_val, y_train, y_val = train_test_split(train_X, train_Y, test_size=0.2, random_state=777)

# 데이터 전처리 - 표준화

In [24]:
# scaler=StandardScaler()
train_X_scale=np.array(x_train)#scaler.fit_transform(x_train)
val_X_scale=np.array(x_val)#scaler.transform(x_val)
test_X_scale=np.array(test_X)#scaler.transform(test_X)

In [25]:
train_X_tensor=torch.FloatTensor(train_X_scale).cuda()
val_X_tensor=torch.FloatTensor(val_X_scale).cuda()
test_X_tensor=torch.FloatTensor(test_X_scale).cuda()

In [26]:
train_Y_tensor=torch.LongTensor(y_train).cuda()

In [27]:
train_X_tensor.shape

In [28]:
train_Y_tensor.max()

# 모델 빌드 & 학습

In [64]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        drop_prob = 0.5 #baseline: 0.3
        self.fc1 = nn.Linear(256, 198,bias=True)
        self.fc2 = nn.Linear(198, 198,bias=True)
        self.fc3 = nn.Linear(198, 198,bias=True)

        self.fc4 = nn.Linear(198, 198)
        self.relu = torch.nn.Sigmoid()
        self.dropout = torch.nn.Dropout(p=drop_prob) #드롭아웃
        ## initialize weight
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_() 
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        
        return x

In [65]:
import torch.optim as optim
net = Net()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [66]:
train_Y_tensor.shape

In [67]:
from sklearn.metrics import accuracy_score
epoch=2000
net=net.cuda()
from tqdm import tqdm

for i in tqdm(range(2000)):
    optimizer.zero_grad()
    # forward + backward + optimize
    outputs = net(train_X_tensor)
    loss = criterion(outputs, train_Y_tensor)
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        val_predict=net(val_X_tensor)
        _, val_predicted = torch.max(val_predict, 1)
        score=accuracy_score(val_predicted.detach().cpu(),y_val)
    if i%100==0:
        print(i," = ",score)


In [68]:
score

In [69]:
train_Y_tensor.shape

# Test 데이터 예측

In [70]:
predict=net(test_X_tensor)
_, predicted = torch.max(predict, 1)
predicted=predicted.detach().cpu().numpy()

In [71]:
pred_={"label":predicted}
submission = pd.DataFrame(data=pred_)
submission.index = [i for i in range(test_X_tensor.shape[0])]
submission.index.name = 'id'


submission.to_csv('kiy-DNN_Pred4.csv', index=True) 

In [72]:
submission

In [63]:
predict